In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "ruslanmv/Medical-Llama3-8B"
device_map = 'auto'

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load model and tokenizer with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,
    device_map=device_map
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [27]:
def askme(question):
    # Define the system instruction
    sys_message = '''
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''

    # Combine system message and user question
    prompt = f"{sys_message}\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate response with strict constraints
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,  # Limit to a short answer
        no_repeat_ngram_size=2,  # Avoid repetition
        repetition_penalty=1.5,  # Penalize repetitive content
        early_stopping=True,  # Stop at the most likely endpoint
        temperature=0.5,  # Reduce randomness for focused answers
        top_p=0.9,  # Use nucleus sampling for diversity
        pad_token_id=tokenizer.eos_token_id, #avoid warnings for padding
    )

    # Decode and clean up the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Extract only the answer part (after "Answer:")
    answer = response.split("Answer:")[-1].strip()
    return answer



In [48]:
# Define a question
question = "What disease is treated with Metformin?"

# Get the AI's direct answer
response = askme(question)
print("AI Answer:", response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AI Answer: Diabetes mellitus type 2

Explanation:
Metforminin (brand name Glucophage) belongs in class called biguanides which act by reducing glucose production from liver cells while increasing insulin sensitivity at muscle cell level leading up improved glycemic control among patients suffering diabetes Mellitius Type II. It has been shown effective when used alone or combined alongside other antidiabetic drugs such as sulfonylureas, thiazolidinediones etc.. However its use must always take into account individual patient characteristics like age gender body mass index presence / absence comorbidities prior history treatment response side effects tolerability cost effectiveness considerations before making any decision regarding prescription.

Note : This content was generated automatically through natural language processing techniques applied over publicly available resources online


In [29]:
#evaluating this model

In [44]:
test_data = [
    {
        "question": "What disease is treated with Metformin?",
        "reference_answer": "Metformin is used to treat Type 2 Diabetes."
    },
    {
        "question": "What disease is treated with Levothyroxine?",
        "reference_answer": "Levothyroxine is used to treat Hypothyroidism."
    },
    {
        "question": "What disease is treated with Atorvastatin?",
        "reference_answer": "Atorvastatin is used to treat High Cholesterol."
    },
    {
        "question": "What disease is treated with Albuterol?",
        "reference_answer": "Albuterol is used to treat Asthma."
    },
    {
        "question": "What disease is treated with Amoxicillin?",
        "reference_answer": "Amoxicillin is used to treat Bacterial Infections."
    },
    {
        "question": "What disease is treated with Lisinopril?",
        "reference_answer": "Lisinopril is used to treat Hypertension."
    },
    {
        "question": "What disease is treated with Omeprazole?",
        "reference_answer": "Omeprazole is used to treat Acid Reflux or GERD."
    },
    {
        "question": "What disease is treated with Fluoxetine?",
        "reference_answer": "Fluoxetine is used to treat Depression."
    },
    {
        "question": "What disease is treated with Warfarin?",
        "reference_answer": "Warfarin is used to treat and prevent Blood Clots."
    },
    {
        "question": "What disease is treated with Insulin?",
        "reference_answer": "Insulin is used to treat Diabetes (Type 1 and advanced Type 2)."
    }
]

